In [2]:
import sys
import os
import astropy.io.fits as fits
import math
import os.path
from astropy.visualization import astropy_mpl_style
from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits
# import matplotlib as plt
import os.path
import matplotlib.pyplot as plt
import operator
from matplotlib.colors import LogNorm
from photutils.datasets import make_4gaussians_image
from photutils import centroid_com, centroid_1dg, centroid_2dg
import numpy as np
from astropy.visualization import PercentileInterval
from photutils import CircularAperture
from photutils import aperture_photometry
from astropy.io import ascii
from astropy.table import Table, Column, MaskedColumn

In [3]:
# definitions
def slicing_im(npix,box):
    npix = npix #total pixels
    box = box #changes the display of arrays
    result= (npix - box)/2
#     print(result)
#     print(result)
    if result % 1 == 0:
        end_num = int(result)
        strt_num = end_num
        new_st = strt_num
        new_end = npix - end_num
    else:
#         end_num = int(round(result,0))
        end_num = result +0.5
        strt_num = end_num - 1
        new_st = strt_num 
        new_end = npix - end_num
#     print("total pixels is: ",npix,'\nLeft side start by:',new_st,"\nRight side end by:",new_end)
    strt = int(new_st)
    end = int(new_end)
    return strt, end

def scale_image(scale,time, data_image):
    scale_num = scale
    interval = PercentileInterval(scale_num)
    interval.get_limits(data_image[time,:,:])
    scale_array = interval(data_image[time,:,:])
    return scale_array 

def diff_arr(array):
    new_array = []
    for i in range(len(array)):
        result = array[i]-i-1
        new_array.append(result)
    return new_array

def aperture_sum(radius, data, val_x, val_y):
    R = radius
    val_2dg_x = val_x
    val_2dg_y = val_y
    # for positions we will take the 2dg Gaussian outputs.
    positions = [(val_2dg_x, val_2dg_y)]
    apertures = CircularAperture(positions, r=R)
    # print(apertures)
    phot_table = aperture_photometry(data,apertures)
    return phot_table['aperture_sum']
# definitions
def slicing_im(npix,box):
    npix = npix #total pixels
    box = box #changes the display of arrays
    result= (npix - box)/2
#     print(result)
#     print(result)
    if result % 1 == 0:
        end_num = int(result)
        strt_num = end_num
        new_st = strt_num
        new_end = npix - end_num
    else:
#         end_num = int(round(result,0))
        end_num = result +0.5
        strt_num = end_num - 1
        new_st = strt_num 
        new_end = npix - end_num
#     print("total pixels is: ",npix,'\nLeft side start by:',new_st,"\nRight side end by:",new_end)
    strt = int(new_st)
    end = int(new_end)
    return strt, end

def scale_image(scale,time, data_image):
    scale_num = scale
    interval = PercentileInterval(scale_num)
    interval.get_limits(data_image[time,:,:])
    scale_array = interval(data_image[time,:,:])
    return scale_array

def org_dict(dict_name,str_num_name,com_x,com_y,dg_x,dg_y):
    start_ind =0 
    end_ind = 15
    str_arr_list = ["cen_com_arr_x", "cen_com_arr_y","cen_2dg_arr_x","cen_2dg_arr_y"]
    dict_cen = dict_name
# puts each into a seperate array of 15 indexes
    for i in range(0,64):
        dict_cen[str_arr_list[0]+'_'+str(i)] = com_x[start_ind:end_ind]
        dict_cen[str_arr_list[1]+'_'+str(i)] = com_y[start_ind:end_ind]
        dict_cen[str_arr_list[2]+'_'+str(i)] = dg_x[start_ind:end_ind]
        dict_cen[str_arr_list[3]+'_'+str(i)] = dg_y[start_ind:end_ind]
        start_ind = start_ind + 15
        end_ind = end_ind+15
#creates an array of numbered names.
    str_num_arr=str_num_name
    for j in range(0,64):
        str_name_1 = str_arr_list[0]+'_'+str(j)
        str_name_2 = str_arr_list[1]+'_'+str(j)
        str_name_3 = str_arr_list[2]+'_'+str(j)
        str_name_4 = str_arr_list[3]+'_'+str(j)
        str_num_arr.append(str_name_1)
        str_num_arr.append(str_name_2)
        str_num_arr.append(str_name_3)
        str_num_arr.append(str_name_4)
        
def time_arr_func(time_arr,ind_time_arr,frame_t,bmjd_obs):
    frameday = frame_t/(3600*24)
    shift_bmjd = bmjd_obs + 0.5* frameday
    list_bmjd=[shift_bmjd]
    time_arr.append(shift_bmjd)
    ind_time_arr.append(shift_bmjd)
    # Adding onto array.
    for j in range(63):
        value = list_bmjd[0] + (j+1)*frameday
        ind_time_arr.append(value)
        time_arr.append(value)

In [7]:
tot_dir_bcd = ['/Users/melaniapena/Rsrch/Luhman_16_Research/bcd_files/only_bcd_part_1',
           '/Users/melaniapena/Rsrch/Luhman_16_Research/bcd_files/only_bcd_part_2',
           '/Users/melaniapena/Rsrch/Luhman_16_Research/bcd_files/only_bcd_part_3',
           '/Users/melaniapena/Rsrch/Luhman_16_Research/bcd_files/only_bcd_part_4',
           '/Users/melaniapena/Rsrch/Luhman_16_Research/bcd_files/only_bcd_part_5',
           '/Users/melaniapena/Rsrch/Luhman_16_Research/bcd_files/only_bcd_part_6-1',
           '/Users/melaniapena/Rsrch/Luhman_16_Research/bcd_files/only_bcd_part_6-2',
           '/Users/melaniapena/Rsrch/Luhman_16_Research/bcd_files/only_bcd_part_7',]

str_arr_list = ["cen_com_arr_x", "cen_com_arr_y","cen_2dg_arr_x","cen_2dg_arr_y"]
col_1='Ap Phot'
col_2='bmjd'

part_num=2

if part_num==6:
    ori_dir='/Users/melaniapena/Rsrch/Luhman_16_Research/ap_phot_ascii/ap_ph_prt_6-1_sample/'
elif part_num==7:
    ori_dir='/Users/melaniapena/Rsrch/Luhman_16_Research/ap_phot_ascii/ap_ph_prt_6-2_sample/'
elif part_num==8:
    ori_dir='/Users/melaniapena/Rsrch/Luhman_16_Research/ap_phot_ascii/ap_ph_prt_7/'
else:
    ori_dir='/Users/melaniapena/Rsrch/Luhman_16_Research/ap_phot_ascii/ap_ph_prt_'+str(part_num)+'/'

print(ori_dir)

/Users/melaniapena/Rsrch/Luhman_16_Research/ap_phot_ascii/ap_ph_prt_2/


In [8]:
phot_ap_arr =[]
time_arr = []
cen_com_arr_x=[]
cen_com_arr_y=[]

m = 0

scale_num = 100
npix = 32
start_i = 3
steps_i = 2

start_ind =0 
end_ind = 15

if part_num == 2 or part_num == 4 or part_num == 5:
    r_1 = 6
    r_2 = 12
    r_3 = 14
else:
    r_1 = 8
    r_2 = 13
    r_3 = 23
print(r_1,r_2,r_3)

6 12 14


In [ ]:
for filename in os.listdir(tot_dir_bcd[part_num-1]):
    if filename.endswith(".fits"): 
#         dir_bcd ='/Users/melaniapena/Rsrch/Luhman_16_Research/bcd_files/only_bcd_part_1/SPITZER_I1_50037760_0000_0000_2_bcd.fits'
        dir_bcd = os.path.join(tot_dir_bcd[part_num-1],filename)
        fits_file = fits.open(dir_bcd)
        fits_data = get_pkg_data_filename(dir_bcd)
        list_Primary = fits_file[0]
        image_data= fits.getdata(fits_data, ext=0)
        
        frame_t = fits_file[0].header['FRAMTIME']
        bmjd_obs = fits_file[0].header['BMJD_OBS']
#         print('bmjd observation:', bmjd_obs)
        flux_conv= fits_file[0].header['FLUXCONV']
#         print('Flux Conversion',flux_conv)
        gain = fits_file[0].header['GAIN']
#         print('Gain',gain)
        exp_time= fits_file[0].header['EXPTIME']
#         print('Exp-time', exp_time)
        conversion_factor = gain*exp_time/flux_conv
#         print(conversion_factor,'\n')
        counts_image = image_data * conversion_factor
        
        num = 0
        centroid_com_arr_x = []
        centroid_com_arr_y = []
        centroid_2dG_arr_x = []
        centroid_2dG_arr_y = []
        
        centroid_com_arr_x1=[]
        centroid_com_arr_y1=[]
        
#         scale_num = 100
#         npix = 32
#         start_i = 3
#         steps_i = 2
#         -----------------------------------------------
#         Finds the Centroids from the various sliced images

        for time_i in range(0,64):        
            for i_box in range(start_i,npix,steps_i):
                strt, end = slicing_im(npix,i_box)
                interval = PercentileInterval(scale_num) ###
                interval.get_limits(counts_image[time_i,strt:end,strt:end])
                scale_array = interval(counts_image[time_i,strt:end,strt:end])
                sliced_image = counts_image[0,strt:end,strt:end] 
                result = scale_array
                if i_box < 3:
                    print('too small of an image to calculate\n')
                elif i_box >= 3:
                    x1, y1 = centroid_com(result)
                    x2, y2 = centroid_2dg(result)
                    centroid_com_arr_x.append(x1)
                    centroid_com_arr_y.append(y1)
                    centroid_2dG_arr_x.append(x2)
                    centroid_2dG_arr_y.append(y2)
#                     print(i_box,strt,end)
#                     print('Box Size: ', i_box,'x',i_box)
#                     print('COM: ',x1,y1)
#                     print('2dG: ',x2,y2)
                    if i_box == 15:
                        centroid_com_arr_x1.append(x1)
                        centroid_com_arr_y1.append(y1)
                        cen_com_arr_x.append(x1)
                        cen_com_arr_y.append(y1)
                        
                    num = num+1
#                     print(num,time_i,'\n')
#                     plt.imshow(result)
#                     plt.show()

#       the empty array and dictionary will help organize the list of centroids thats
#       taken out of each file.
        str_num_arr=[]      
        dict_cen = {}
#         The line below is a define function (found in previous cell)
#         that organizes the centroids.
        org_dict(dict_cen,str_num_arr,centroid_com_arr_x,centroid_com_arr_y,centroid_2dG_arr_x,centroid_2dG_arr_y)
        
#         -----------------------------------------------
#         Aperture Photometry

#         focuses on the indiviual array of the photo data.
        ind_photometry_arr =[]
        h_2dg = 2 ##starting index for 2dG
        print(org_dict)
        for l in range(0,64):
            aperture_1 = aperture_sum(r_1,counts_image[l,:,:], dict_cen[str_num_arr[h_2dg]][14], dict_cen[str_num_arr[h_2dg+1]][14]) 
            aperture_2 = aperture_sum(r_2,counts_image[l,:,:], dict_cen[str_num_arr[h_2dg]][14], dict_cen[str_num_arr[h_2dg+1]][14])
            aperture_3 = aperture_sum(r_3,counts_image[l,:,:], dict_cen[str_num_arr[h_2dg]][14], dict_cen[str_num_arr[h_2dg+1]][14])
## Area of aperture,
            area_1 = math.pi * r_1**2
## Background in annulus from r_2 to r_3
            background_counts = aperture_3 - aperture_2
## Area of Annulus
            area_2 = math.pi*(r_3**2 - r_2**2) #also known as npix
## subtracting background: 
            backgrnd_subtract = aperture_1 - background_counts*area_1/area_2
            ind_photometry_arr.append(backgrnd_subtract)
            phot_ap_arr.append(backgrnd_subtract)
            h_2dg = h_2dg+4

#         -----------------------------------------------
#         Time array

        ind_time_arr = []
        time_arr_func(time_arr,ind_time_arr,frame_t,bmjd_obs)

#         -----------------------------------------------
#         Saves it into an ascii array

#         print(num)
#         plt.plot(ind_time_arr,photometry_arr)
#         plt.plot(photometry_arr)

        if part_num==6:
            str_m='_part_6-1'
        elif part_num==7:
            str_m='_part_6-2'
        elif part_num==8:
            str_m='_part_7'
        else:
            str_m='_part_'+str(part_num)
        
        word=ori_dir+'bcdfits_'+str(m)+str_m+'.txt'
        print(m)
        m+=1
#         plt.savefig(word)
        ind_arr_table = Table([ind_photometry_arr,ind_time_arr,centroid_com_arr_x1,centroid_com_arr_y1], names=[col_1,col_2,'x_com_cen','y_com_cen'])
        ascii.write(ind_arr_table, word, format='tab', fast_writer=False) 
#         plt.show()    
#         -----------------------------------------------

    else:
        continue

if part_num==6:
    str_m='part_6-1'
elif part_num==7:
    str_m='part_6-2'
elif part_num==8:
    str_m='part_7'
else:
    str_m='part_'+str(part_num)


word=ori_dir+'bcdfits_'+str_m+'.txt'
Tot_arr_table = Table([phot_ap_arr,time_arr,cen_com_arr_x,cen_com_arr_y], names=[col_1,col_2,'x_com_cen','y_com_cen'])
ascii.write(Tot_arr_table, word, format='tab', fast_writer=False) 
        

/anaconda3/envs/astroconda/lib/python3.5/site-packages/skimage/measure/_moments.py:258: UserWarning: Using deprecated 2D-only, xy-coordinate interface to moments_central. This interface will be removed in scikit-image 0.16. Use moments_central(image, center=(cr, cc), order=3).
  warn(message)
